# パイプラインを公開する

[前のラボ](labdocs/Lab06A.md)では、パイプラインを作成しました。次に、サービスとして公開します。

## ワークスペースに接続する

まず、Azure ML SDK を使用してワークスペースに接続する必要があります。

> **注**: 前回の演習を完了してから Azure サブスクリプションとの認証済みセッションの有効期限が切れている場合は、再認証を求めるメッセージが表示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存した構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## パイプラインを公開する

パイプラインを作成してテストした後、REST サービスとして公開できます。[前のラボ](labdocs/Lab06A.md)でパイプラインを実行したので、その実行への参照を取得し、それを使用してパイプラインを公開できます。

In [ ]:
# パイプラインの最新の実行を取得する
experiment_name = 'diabetes-training-pipeline'
pipeline_experiment = ws.experiments.get(experiment_name)
pipeline_run = list(pipeline_experiment.get_runs())[0]

# 実行からパイプラインを公開する
published_pipeline = pipeline_run.publish_pipeline(
    name="Diabetes_Training_Pipeline", description="Trains diabetes model", version="1.0")

published_pipeline

公開されたパイプラインにはエンドポイントがあり、[Azure Machine Learning Studio](https://ml.azure.com) の**エンドポイント** ページ (**パイプライン エンドポイント** タブ) に表示されます。また、公開されたパイプライン オブジェクトのプロパティとして URI を見つけることもできます。

In [ ]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

## パイプライン エンドポイントを使用する

エンドポイントを使用するには、クライアント アプリケーションが HTTP 経由で REST 呼び出しを行う必要があります。この要求は認証される必要があるため、Authorization ヘッダーが必要です。実際のアプリケーションには、認証に使用するサービス プリンシパルが必要ですが、これをテストするには、現在の接続から Azure ワークスペースへの Authorization ヘッダーを使用します。これは、次のコードを使用して取得できます。

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print("Authentication header ready.")

これで、REST インターフェイスを呼び出す準備ができました。パイプラインは非同期で実行されるため、識別子を取得します。実行中のパイプライン実験を追跡するために使用できます。

In [ ]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": experiment_name})
run_id = response.json()["Id"]
run_id

実行 ID があるため、**RunDetails** ウィジェットを使用して、実行中の実験を表示できます。

> **注**: 各ステップは出力の再利用を許可するように構成されているため、パイプラインは迅速に完了するはずです。これは、主に利便性とこのコースの時間を節約するために行われました。実際には、データが変更された場合に備えて、最初のステップを毎回実行し、ステップ 1 の出力が変更された場合にのみ後続のステップをトリガーする必要があります。

In [ ]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments[experiment_name], run_id)
RunDetails(published_pipeline_run).show()

## パイプラインをスケジュールする

たとえば、糖尿病患者の診療所が毎週新しいデータを収集し、データセットに追加したとします。パイプラインを毎週実行して、新しいデータでモデルを再トレーニングできます。

In [ ]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

# 毎週月曜日の 00:00 UTC にパイプラインを送信する
recurrence = ScheduleRecurrence(frequency="Week", interval=1, week_days=["Monday"], time_of_day="00:00")
weekly_schedule = Schedule.create(ws, name="weekly-diabetes-training", 
                                  description="Based on time",
                                  pipeline_id=published_pipeline.id, 
                                  experiment_name=experiment_name, 
                                  recurrence=recurrence)
print('Pipeline scheduled.')

以下のように、ワークスペースで定義されているスケジュールを取得できます。

In [ ]:
schedules = Schedule.list(ws)
schedules

次のように最新の実行を確認できます。

In [ ]:
pipeline_experiment = ws.experiments.get(experiment_name)
latest_run = list(pipeline_experiment.get_runs())[0]

latest_run.get_details()

> **詳細情報**: パイプラインのスケジュール設定の詳細については、[ドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-schedule-pipelines)を参照してください。